In [1]:
import pandas as pd
import numpy as np
import warnings
import sys, os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

sys.path.append(os.path.abspath('..'))

label_path = '../../data/labels/full_non_transit_radar_labels.csv'
image_path = 'nontransit_track_images/'

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

import warnings
warnings.filterwarnings("ignore", message="Failed to load image Python extension")

Using device: mps


#### CNNBaseline

In [2]:
BATCH_SIZE = 32
NUM_HIDDEN_1 = 32
NUM_HIDDEN_2 = 64
NUM_HIDDEN_3 = 128
IN_CHANNELS = 3

from core.DICT import TYPE2NUM
NUM_CLASSES = len(TYPE2NUM)

from dataloader_imgs import get_type_datasets
train_loader, val_loader, test_loader = get_type_datasets(label_path, image_path, batch_size = BATCH_SIZE)

In [3]:
#Check dataloader
for next_batch in train_loader:
    print(next_batch[0].shape)
    print(next_batch[1].shape)
    print(next_batch[1])
    break

print(len(train_loader))
print(len(val_loader))
print(len(test_loader))


torch.Size([32, 3, 224, 224])
torch.Size([32])
tensor([0, 3, 6, 0, 5, 2, 3, 5, 0, 3, 5, 5, 6, 6, 0, 1, 6, 5, 0, 0, 2, 5, 0, 5,
        6, 0, 5, 5, 0, 0, 0, 5])
43
6
6


In [4]:
from cnn_models import CNNBaseline
from cnn_training import train_model, save_model

cnn_baseline_1 = CNNBaseline(IN_CHANNELS, NUM_HIDDEN_1, NUM_HIDDEN_2, NUM_HIDDEN_3, NUM_CLASSES).to(device)
optimizer = optim.Adam(cnn_baseline_1.parameters(), lr=1e-4)
model_name = "nontransit_cnn_baseline_best_10"

train_config = {
    "model": cnn_baseline_1,
    "train_loader" : train_loader,
    "val_loader" : val_loader,
    "optimizer" : optimizer,
    "device" : device,
    "num_epochs" : 10,
    "scheduler": None
}

#train and save model
TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_model_wts, best_val_loss = train_model(**train_config)
save_model(TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_val_loss, best_model_wts, model_name)


Epoch 1/10
Train Loss: 1.6519 | Train Acc: 0.3114
Val   Loss: 1.6406 | Val   Acc: 0.3882


Epoch 2/10
Train Loss: 1.6283 | Train Acc: 0.3136
Val   Loss: 1.6324 | Val   Acc: 0.3882


Epoch 3/10
Train Loss: 1.6166 | Train Acc: 0.3414
Val   Loss: 1.6481 | Val   Acc: 0.3882


Epoch 4/10
Train Loss: 1.5818 | Train Acc: 0.3392
Val   Loss: 1.5078 | Val   Acc: 0.4118


Epoch 5/10
Train Loss: 1.4885 | Train Acc: 0.4057
Val   Loss: 1.4051 | Val   Acc: 0.4412


Epoch 6/10
Train Loss: 1.3880 | Train Acc: 0.4496
Val   Loss: 1.3428 | Val   Acc: 0.4765


Epoch 7/10
Train Loss: 1.3337 | Train Acc: 0.4766
Val   Loss: 1.3752 | Val   Acc: 0.4706


Epoch 8/10
Train Loss: 1.2447 | Train Acc: 0.5161
Val   Loss: 1.3336 | Val   Acc: 0.4941


Epoch 9/10
Train Loss: 1.1928 | Train Acc: 0.5380
Val   Loss: 1.3766 | Val   Acc: 0.4647


Epoch 10/10
Train Loss: 1.1354 | Train Acc: 0.5475
Val   Loss: 1.3859 | Val   Acc: 0.4412


In [5]:
#Retrain for further epochs
cnn_baseline_1 = CNNBaseline(IN_CHANNELS, NUM_HIDDEN_1, NUM_HIDDEN_2, NUM_HIDDEN_3, NUM_CLASSES).to(device)

checkpoint_path = f"models/{model_name}.pth"
cnn_baseline_1.load_state_dict(torch.load(checkpoint_path))
optimizer = optim.Adam(cnn_baseline_1.parameters(), lr=1e-4)
model_name = "nontransit_cnn_baseline_best_20"

train_config_resume = {
    "model": cnn_baseline_1,
    "train_loader": train_loader,
    "val_loader": val_loader,
    "optimizer": optimizer,
    "device": device,
    "num_epochs": 10,
    "scheduler": None
}

# 5. Resume training
train_loss_2, train_acc_2, val_loss_2, val_acc_2, best_model_wts_2, best_val_loss_2 = train_model(**train_config_resume)
save_model(TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_val_loss, best_model_wts, model_name)


Epoch 1/10
Train Loss: 1.1960 | Train Acc: 0.5380
Val   Loss: 1.3280 | Val   Acc: 0.5000


Epoch 2/10
Train Loss: 1.1490 | Train Acc: 0.5643
Val   Loss: 1.3409 | Val   Acc: 0.4941


Epoch 3/10
Train Loss: 1.1040 | Train Acc: 0.5614
Val   Loss: 1.3498 | Val   Acc: 0.4588


Epoch 4/10
Train Loss: 1.0387 | Train Acc: 0.6001
Val   Loss: 1.3564 | Val   Acc: 0.4588


Epoch 5/10
Train Loss: 1.0001 | Train Acc: 0.6148
Val   Loss: 1.3798 | Val   Acc: 0.4824


Epoch 6/10
Train Loss: 0.9397 | Train Acc: 0.6330
Val   Loss: 1.4723 | Val   Acc: 0.4706


Epoch 7/10
Train Loss: 0.8614 | Train Acc: 0.6681
Val   Loss: 1.5020 | Val   Acc: 0.4706


Epoch 8/10
Train Loss: 0.8042 | Train Acc: 0.7003
Val   Loss: 1.5354 | Val   Acc: 0.4529


Epoch 9/10
Train Loss: 0.7627 | Train Acc: 0.7164
Val   Loss: 1.5814 | Val   Acc: 0.4529


Epoch 10/10
Train Loss: 0.6715 | Train Acc: 0.7602
Val   Loss: 1.7188 | Val   Acc: 0.4235


### Resnet

In [7]:
from cnn_models import get_resnet50_classifier

resnet50 = get_resnet50_classifier(NUM_CLASSES).to(device)
optimizer = optim.Adam(resnet50.parameters(), lr=1e-3)
model_name = "nontransit_resnet50_best_10"

train_config = {
    "model": resnet50,
    "train_loader" : train_loader,
    "val_loader" : val_loader,
    "optimizer" : optimizer, 
    "device" : device,
    "num_epochs" : 10,
    "scheduler": None
}

#train loop
TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_model_wts, best_val_loss = train_model(**train_config)
save_model(TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_val_loss, best_model_wts, model_name)

Epoch 1/10
Train Loss: 1.6182 | Train Acc: 0.3516
Val   Loss: 1.5330 | Val   Acc: 0.3941


Epoch 2/10
Train Loss: 1.4614 | Train Acc: 0.4496
Val   Loss: 1.4333 | Val   Acc: 0.4412


Epoch 3/10
Train Loss: 1.3859 | Train Acc: 0.4817
Val   Loss: 1.3801 | Val   Acc: 0.4471


Epoch 4/10
Train Loss: 1.3317 | Train Acc: 0.5080
Val   Loss: 1.3430 | Val   Acc: 0.4765


Epoch 5/10
Train Loss: 1.2863 | Train Acc: 0.5278
Val   Loss: 1.3487 | Val   Acc: 0.5059


Epoch 6/10
Train Loss: 1.2511 | Train Acc: 0.5270
Val   Loss: 1.3289 | Val   Acc: 0.4882


Epoch 7/10
Train Loss: 1.2039 | Train Acc: 0.5541
Val   Loss: 1.3115 | Val   Acc: 0.5235


Epoch 8/10
Train Loss: 1.1840 | Train Acc: 0.5651
Val   Loss: 1.2972 | Val   Acc: 0.5294


Epoch 9/10
Train Loss: 1.1616 | Train Acc: 0.5877
Val   Loss: 1.2856 | Val   Acc: 0.5235


Epoch 10/10
Train Loss: 1.1359 | Train Acc: 0.5892
Val   Loss: 1.2887 | Val   Acc: 0.5235


In [8]:
resnet50 = get_resnet50_classifier(NUM_CLASSES).to(device)

checkpoint_path = "models/nontransit_resnet50_best_10.pth"
resnet50.load_state_dict(torch.load(checkpoint_path))
optimizer = optim.Adam(resnet50.parameters(), lr=1e-4)

train_config_resume = {
    "model": resnet50,
    "train_loader": train_loader,
    "val_loader": val_loader,
    "optimizer": optimizer,
    "device": device,
    "num_epochs": 10,
    "scheduler": None
}

model_name = "nontransit_resnet50_best_20"
train_loss_2, train_acc_2, val_loss_2, val_acc_2, best_model_wts_2, best_val_loss_2 = train_model(**train_config_resume)
save_model(TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_val_loss, best_model_wts, model_name)

Epoch 1/10
Train Loss: 1.1528 | Train Acc: 0.5753
Val   Loss: 1.2993 | Val   Acc: 0.5118


Epoch 2/10
Train Loss: 1.1299 | Train Acc: 0.5848
Val   Loss: 1.2809 | Val   Acc: 0.5176


Epoch 3/10
Train Loss: 1.1386 | Train Acc: 0.5914
Val   Loss: 1.2714 | Val   Acc: 0.5529


Epoch 4/10
Train Loss: 1.1334 | Train Acc: 0.5753
Val   Loss: 1.2895 | Val   Acc: 0.5176


Epoch 5/10
Train Loss: 1.1281 | Train Acc: 0.6031
Val   Loss: 1.2775 | Val   Acc: 0.5294


Epoch 6/10
Train Loss: 1.1292 | Train Acc: 0.5841
Val   Loss: 1.2932 | Val   Acc: 0.5176


Epoch 7/10
Train Loss: 1.1258 | Train Acc: 0.6001
Val   Loss: 1.2634 | Val   Acc: 0.5412


Epoch 8/10
Train Loss: 1.1250 | Train Acc: 0.5892
Val   Loss: 1.2695 | Val   Acc: 0.5235


Epoch 9/10
Train Loss: 1.1079 | Train Acc: 0.6001
Val   Loss: 1.2731 | Val   Acc: 0.5235


Epoch 10/10
Train Loss: 1.1165 | Train Acc: 0.6016
Val   Loss: 1.2771 | Val   Acc: 0.5353
